# 模型训练

In [1]:
from PIL import Image  
# 增加PIL的解压炸弹限制
Image.MAX_IMAGE_PIXELS = None 

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## CUDA版本配置

In [3]:
import os

# 添加cuda环境变量
os.environ["CUDA_PATH"] = r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8"

# 添加cudnn环境变量
cudnn_paths = [
             r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin",
             r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\libnvvp",
             r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\include",
             r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\lib",
            ]
for cudnn_path in cudnn_paths:
    if cudnn_path not in os.environ["PATH"]:
        os.environ["PATH"] = cudnn_path + os.pathsep + os.environ["PATH"]

In [4]:
print(os.environ["PATH"])
print(os.environ["CUDA_PATH"])
!nvcc -V

C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\include;c:\Users\Rooki\.conda\envs\rtdetr;C:\Users\Rooki\.conda\envs\rtdetr;C:\Users\Rooki\.conda\envs\rtdetr\Library\mingw-w64\bin;C:\Users\Rooki\.conda\envs\rtdetr\Library\usr\bin;C:\Users\Rooki\.conda\envs\rtdetr\Library\bin;C:\Users\Rooki\.conda\envs\rtdetr\Scripts;C:\Users\Rooki\.conda\envs\rtdetr\bin;C:\ProgramData\miniconda3\condabin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\libnvvp;.;C:\windows\system32;C:\windows;C:\windows\System32\Wbem;C:\windows\System32\WindowsPowerShell\v1.0;C:\windows\System32\OpenSSH;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR;C:\Users\Administrator\AppData\Local\Microsoft\WindowsApps;C:\Program Files\HP\OMEN-Broadcast\Common;C:\Program F

In [5]:
!nvidia-smi

Fri Apr 18 14:38:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.19                 Driver Version: 561.19         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   43C    P8              2W /   75W |     412MiB /   8188MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
import torch

print(f"torch.cuda.is_available(): {torch.cuda.is_available()}")
print(f"torch.cuda.get_device_name(0): {torch.cuda.get_device_name(0)}")


torch.cuda.is_available(): True
torch.cuda.get_device_name(0): NVIDIA GeForce RTX 4060 Laptop GPU


## 启动训练

In [7]:
# 导入必要的库
import os 
import sys 
# 将项目根目录添加到系统路径
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import argparse

# 导入自定义模块
from src.misc import dist_utils  # 分布式训练相关工具
from src.core import YAMLConfig, yaml_utils  # YAML配置文件处理工具
from src.solver import TASKS  

In [8]:
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
root_dir

'c:\\Users\\Rooki\\Desktop\\AI\\CV\\RT-DETR\\rtdetrv2_pytorch'

In [9]:
# 在jupyter中直接设置参数
class Args:
    def __init__(self):
        self.config = f'{root_dir}/configs/rtdetrv2/rtdetrv2_r50vd_6x_chengdu.yml'  # 配置文件路径
        self.resume = None  # 恢复训练的检查点路径
        self.tuning = None  # 微调模式的预训练模型路径
        self.device = None  # 指定设备
        self.seed = None  # 随机种子,用于实验可重复性
        self.use_amp = False  # 是否使用混合精度训练
        self.output_dir = None  # 输出目录
        self.summary_dir = None  # tensorboard日志目录
        self.test_only = False  # 是否只执行测试
        self.update = None  # YAML配置文件的更新参数
        self.print_method = 'builtin'  # 打印方法
        self.print_rank = 0  # 打印进程的rank id
        self.local_rank = None  # 本地进程的rank id

args = Args()

In [10]:
# 设置分布式训练环境
dist_utils.setup_distributed(args.print_rank, args.print_method, seed=args.seed)

# 确保不会同时进行微调和恢复训练
assert not all([args.tuning, args.resume]), \
    'Only support from_scrach or resume or tuning at one time'

# 解析命令行更新参数并与其他参数合并
update_dict = yaml_utils.parse_cli(args.update)
update_dict.update({k: v for k, v in args.__dict__.items() \
    if k not in ['update', ] and v is not None})

# 创建配置对象并打印配置信息
cfg = YAMLConfig(args.config, **update_dict)
print('cfg: ', cfg.__dict__)

Not init distributed mode.
cfg:  {'task': 'detection', '_model': None, '_postprocessor': None, '_criterion': None, '_optimizer': None, '_lr_scheduler': None, '_lr_warmup_scheduler': None, '_train_dataloader': None, '_val_dataloader': None, '_ema': None, '_scaler': None, '_train_dataset': None, '_val_dataset': None, '_collate_fn': None, '_evaluator': None, '_writer': None, 'num_workers': 0, 'batch_size': None, '_train_batch_size': None, '_val_batch_size': None, '_train_shuffle': None, '_val_shuffle': None, 'resume': None, 'tuning': None, 'epoches': 100, 'last_epoch': -1, 'use_amp': False, 'use_ema': True, 'ema_decay': 0.9999, 'ema_warmups': 2000, 'sync_bn': True, 'clip_max_norm': 0.1, 'find_unused_parameters': False, 'seed': None, 'print_freq': 100, 'checkpoint_freq': 1, 'output_dir': './output/rtdetrv2_r50vd_6x_chengdu', 'summary_dir': None, 'device': '', 'yaml_cfg': {'task': 'detection', 'evaluator': {'type': 'CocoEvaluator', 'iou_types': ['bbox']}, 'num_classes': 17, 'remap_mscoco_ca

In [11]:
cfg.yaml_cfg

{'task': 'detection',
 'evaluator': {'type': 'CocoEvaluator', 'iou_types': ['bbox']},
 'num_classes': 17,
 'remap_mscoco_category': True,
 'train_dataloader': {'type': 'DataLoader',
  'dataset': {'type': 'CocoDetection',
   'img_folder': '\\\\?\\C:\\Users\\Rooki\\Desktop\\AI\\CV\\RT-DETR\\chengdu_dataset\\train_rename',
   'ann_file': '\\\\?\\C:\\Users\\Rooki\\Desktop\\AI\\CV\\RT-DETR\\chengdu_dataset\\train_rename\\_annotations.coco.json',
   'return_masks': False,
   'transforms': {'type': 'Compose',
    'ops': None,
    'policy': {'name': 'stop_epoch',
     'epoch': 71,
     'ops': ['RandomPhotometricDistort', 'RandomZoomOut', 'RandomIoUCrop']}}},
  'shuffle': True,
  'num_workers': 4,
  'drop_last': True,
  'collate_fn': {'type': 'BatchImageCollateFuncion',
   'scales': [480, 512, 544, 576, 608, 640, 640, 640, 672, 704, 736, 768, 800],
   'stop_epoch': 71},
  'total_batch_size': 1},
 'val_dataloader': {'type': 'DataLoader',
  'dataset': {'type': 'CocoDetection',
   'img_folder': '\

In [12]:
cfg.yaml_cfg['task']

'detection'

In [13]:
cfg.model

Load PResNet50 state_dict


RTDETR(
  (backbone): PResNet(
    (conv1): Sequential(
      (conv1_1): ConvNormLayer(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (norm): FrozenBatchNorm2d(32, eps=1e-05)
        (act): ReLU(inplace=True)
      )
      (conv1_2): ConvNormLayer(
        (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (norm): FrozenBatchNorm2d(32, eps=1e-05)
        (act): ReLU(inplace=True)
      )
      (conv1_3): ConvNormLayer(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (norm): FrozenBatchNorm2d(64, eps=1e-05)
        (act): ReLU(inplace=True)
      )
    )
    (res_layers): ModuleList(
      (0): Blocks(
        (blocks): ModuleList(
          (0): BottleNeck(
            (branch2a): ConvNormLayer(
              (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (norm): FrozenBatchNorm2d(64, eps=1e-05)
    

In [14]:
from torchinfo import summary
summary(cfg.model.backbone, input_size=(1, 3, 640, 640))

Layer (type:depth-idx)                                       Output Shape              Param #
PResNet                                                      [1, 512, 80, 80]          --
├─Sequential: 1-1                                            [1, 64, 320, 320]         --
│    └─ConvNormLayer: 2-1                                    [1, 32, 320, 320]         --
│    │    └─Conv2d: 3-1                                      [1, 32, 320, 320]         (864)
│    │    └─FrozenBatchNorm2d: 3-2                           [1, 32, 320, 320]         --
│    │    └─ReLU: 3-3                                        [1, 32, 320, 320]         --
│    └─ConvNormLayer: 2-2                                    [1, 32, 320, 320]         --
│    │    └─Conv2d: 3-4                                      [1, 32, 320, 320]         (9,216)
│    │    └─FrozenBatchNorm2d: 3-5                           [1, 32, 320, 320]         --
│    │    └─ReLU: 3-6                                        [1, 32, 320, 320]         

In [15]:
cfg.postprocessor

RTDETRPostProcessor(use_focal_loss=True, num_classes=17, num_top_queries=300)

In [16]:
cfg.epoches = 1

In [17]:
# 根据配置创建对应任务的训练器
solver = TASKS[cfg.yaml_cfg['task']](cfg)

In [18]:
# solver.val()

In [19]:
solver.fit()

Start training
Initial lr: [1e-05, 0.0001, 0.0001]
building train_dataloader with batch_size=1...
loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
building val_dataloader with batch_size=1...
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
number of trainable parameters: 42733395
Total epochs: 1
Epoch: [0]  [   0/5394]  eta: 15:44:48  lr: 0.000000  loss: 113830.6094 (113830.6094)  loss_vfl: 0.0124 (0.0124)  loss_bbox: 8753.0000 (8753.0000)  loss_giou: 2.9093 (2.9093)  loss_vfl_aux_0: 0.0136 (0.0136)  loss_bbox_aux_0: 8753.0000 (8753.0000)  loss_giou_aux_0: 2.9093 (2.9093)  loss_vfl_aux_1: 0.0153 (0.0153)  loss_bbox_aux_1: 8753.0000 (8753.0000)  loss_giou_aux_1: 2.9093 (2.9093)  loss_vfl_aux_2: 0.0137 (0.0137)  loss_bbox_aux_2: 8753.0000 (8753.0000)  loss_giou_aux_2: 2.9093 (2.9093)  loss_vfl_aux_3: 0.0195 (0.0195)  loss_bbox_aux_3: 8753.0000 (8753.0000)  loss_giou_aux_3: 2.9093 (2.9093)  loss_vfl_aux_4: 0.0101 (0.0101

KeyboardInterrupt: 